# Derive Column By Example

One of the more advanced tools in DataPrep is the ability to derive columns by providing examples of desired results and letting DataPrep generate code to achieve the intended derivation.

In [1]:
import azureml.dataprep as dprep

In [2]:
dataflow = dprep.read_csv(path='https://dpreptestfiles.blob.core.windows.net/testfiles/BostonWeather.csv')
df = dataflow.head(10)
df

,DATE,REPORTTPYE,HOURLYDRYBULBTEMPF,HOURLYRelativeHumidity,HOURLYWindSpeed
0,1/1/2015 0:54,FM-15,22,50,10
1,1/1/2015 1:00,FM-12,22,50,10
2,1/1/2015 1:54,FM-15,22,50,10
3,1/1/2015 2:54,FM-15,22,50,11
4,1/1/2015 3:54,FM-15,24,46,13
5,1/1/2015 4:00,FM-12,24,46,13
6,1/1/2015 4:54,FM-15,22,52,15
7,1/1/2015 5:54,FM-15,23,48,17
8,1/1/2015 6:54,FM-15,23,50,14
9,1/1/2015 7:00,FM-12,23,50,14


As you can see, this is a fairly simple file, but let's assume that we need to be able to join this with a dataset where date and time come in a format 'Mar 10, 2018 | 2AM-4AM'.

Let's wrangle the data into the shape we need.

In [3]:
builder = dataflow.builders.derive_column_by_example(source_columns=['DATE'], new_column_name='date_timerange')
builder.add_example(source_data=df.iloc[1], example_value='Jan 1, 2015 12AM-2AM')
builder.preview() # will preview top 10 rows

,DATE,date_timerange
0,1/1/2015 0:54,"Jan 1, 2015 12AM-2AM"
1,1/1/2015 1:00,"Jan 1, 2015 12AM-2AM"
2,1/1/2015 1:54,"Jan 1, 2015 12AM-2AM"
3,1/1/2015 2:54,"Jan 1, 2015 2AM-4AM"
4,1/1/2015 3:54,"Jan 1, 2015 2AM-4AM"
5,1/1/2015 4:00,"Jan 1, 2015 4AM-6AM"
6,1/1/2015 4:54,"Jan 1, 2015 4AM-6AM"
7,1/1/2015 5:54,"Jan 1, 2015 4AM-6AM"
8,1/1/2015 6:54,"Jan 1, 2015 6AM-8AM"
9,1/1/2015 7:00,"Jan 1, 2015 6AM-8AM"


The code above first creates a builder for the derived column by providing an array of source columns to consider ('DATE') and name for the new column to be added.

Then, we provide the first example by passing in the second row (index 1) of the DataFrame printed above and giving an expected value for the derived column.

Finally, we call `builder.preview()` and observe the derived column next to the source column. So far everything seems fine, but we also only see values for the same date "Jan 1, 2015", so we should look at rows further down by passing in the number of rows we want to "skip" from the top.

In [4]:
preview_df = builder.preview(skip=30)
preview_df

,DATE,date_timerange
0,1/1/2015 22:54,"Jan 1, 2015 10PM-12AM"
1,1/1/2015 23:54,"Jan 1, 2015 10PM-12AM"
2,1/1/2015 23:59,"Jan 1, 2015 10PM-12AM"
3,1/2/2015 0:54,"Feb 1, 2015 12AM-2AM"
4,1/2/2015 1:00,"Feb 1, 2015 12AM-2AM"
5,1/2/2015 1:54,"Feb 1, 2015 12AM-2AM"
6,1/2/2015 2:54,"Feb 1, 2015 2AM-4AM"
7,1/2/2015 3:54,"Feb 1, 2015 2AM-4AM"
8,1/2/2015 4:00,"Feb 1, 2015 4AM-6AM"
9,1/2/2015 4:54,"Feb 1, 2015 4AM-6AM"


Here we can see an issue with the generated program: based solely on 1 example we provided above, the derive program chose to parse the date as "Day/Month/Year", which is not what we want in this case.

To fix this issue we need to provide another example.

In [5]:
builder.add_example(source_data=preview_df.iloc[3], example_value='Jan 2, 2015 12AM-2AM')
preview_df = builder.preview(skip=30, count=100)
preview_df

,DATE,date_timerange
0,1/1/2015 22:54,"Jan 1, 2015 10PM-12AM"
1,1/1/2015 23:54,"Jan 1, 2015 10PM-12AM"
2,1/1/2015 23:59,"Jan 1, 2015 10PM-12AM"
3,1/2/2015 0:54,"Jan 2, 2015 12AM-2AM"
4,1/2/2015 1:00,"Jan 2, 2015 12AM-2AM"
5,1/2/2015 1:54,"Jan 2, 2015 12AM-2AM"
6,1/2/2015 2:54,"Jan 2, 2015 2AM-4AM"
7,1/2/2015 3:54,"Jan 2, 2015 2AM-4AM"
8,1/2/2015 4:00,"Jan 2, 2015 4AM-6AM"
9,1/2/2015 4:54,"Jan 2, 2015 4AM-6AM"


Now, rows correctly handle '1/2/2015' as 'Jan 2, 2015', but when we look further down the derived column we can see that values at the end have nothing in derived column. To fix that, we need to provide another example for row 66.

In [6]:
builder.add_example(source_data=preview_df.iloc[66], example_value='Jan 29, 2015 8PM-10PM')
builder.preview(count=100)

,DATE,date_timerange
0,1/1/2015 0:54,"Jan 1, 2015 12AM-2AM"
1,1/1/2015 1:00,"Jan 1, 2015 12AM-2AM"
2,1/1/2015 1:54,"Jan 1, 2015 12AM-2AM"
3,1/1/2015 2:54,"Jan 1, 2015 2AM-4AM"
4,1/1/2015 3:54,"Jan 1, 2015 2AM-4AM"
5,1/1/2015 4:00,"Jan 1, 2015 4AM-6AM"
6,1/1/2015 4:54,"Jan 1, 2015 4AM-6AM"
7,1/1/2015 5:54,"Jan 1, 2015 4AM-6AM"
8,1/1/2015 6:54,"Jan 1, 2015 6AM-8AM"
9,1/1/2015 7:00,"Jan 1, 2015 6AM-8AM"


Everything looks good here. However, we just noticed that it's not quite what we wanted. We forgot to separate date and time range by '|' to generate the format we need.

To fix that, we will add another example. This time, instead of passing in a row from the preview, we just construct a dictionary of column name to value for the source_data parameter.

In [7]:
builder.add_example(source_data={'DATE': '11/11/2015 0:54'}, example_value='Nov 11, 2015 | 12AM-2AM')
builder.preview(count=100)

,DATE,date_timerange
0,1/1/2015 0:54,None
1,1/1/2015 1:00,"Jan 1, 2015 12AM-2AM"
2,1/1/2015 1:54,None
3,1/1/2015 2:54,None
4,1/1/2015 3:54,None
5,1/1/2015 4:00,None
6,1/1/2015 4:54,None
7,1/1/2015 5:54,None
8,1/1/2015 6:54,None
9,1/1/2015 7:00,None


This clearly had negative effects, as now the only rows that have any values in derived column are the ones that match exactly with the examples we have provided.

Let's look at the examples:

In [8]:
examples = builder.list_examples()
examples

,DATE,example,example_id
0,1/1/2015 1:00,"Jan 1, 2015 12AM-2AM",-1
1,1/2/2015 0:54,"Jan 2, 2015 12AM-2AM",-2
2,1/29/2015 20:54,"Jan 29, 2015 8PM-10PM",-3
3,11/11/2015 0:54,"Nov 11, 2015 | 12AM-2AM",-4


Here we can see that we have provided inconsistent examples. To fix the issue, we need to replace the first three examples with correct ones (including '|' between date and time).

We can achieve this by deleting examples that are incorrect (by either passing in example_row from examples DataFrame, or by just passing in example_id value) and then adding new modified examples back.

In [9]:
builder.delete_example(example_id=-1)
builder.delete_example(example_row=examples.iloc[1])
builder.delete_example(example_row=examples.iloc[2])
builder.add_example(examples.iloc[0], 'Jan 1, 2015 | 12AM-2AM')
builder.add_example(examples.iloc[1], 'Jan 2, 2015 | 12AM-2AM')
builder.add_example(examples.iloc[2], 'Jan 29, 2015 | 8PM-10PM')
builder.preview()

,DATE,date_timerange
0,1/1/2015 0:54,"Jan 1, 2015 | 12AM-2AM"
1,1/1/2015 1:00,"Jan 1, 2015 | 12AM-2AM"
2,1/1/2015 1:54,"Jan 1, 2015 | 12AM-2AM"
3,1/1/2015 2:54,"Jan 1, 2015 | 2AM-4AM"
4,1/1/2015 3:54,"Jan 1, 2015 | 2AM-4AM"
5,1/1/2015 4:00,"Jan 1, 2015 | 4AM-6AM"
6,1/1/2015 4:54,"Jan 1, 2015 | 4AM-6AM"
7,1/1/2015 5:54,"Jan 1, 2015 | 4AM-6AM"
8,1/1/2015 6:54,"Jan 1, 2015 | 6AM-8AM"
9,1/1/2015 7:00,"Jan 1, 2015 | 6AM-8AM"


Now this looks correct and we can finally call to_dataflow() on the builder, which would return a dataflow with the desired derived columns added.

In [10]:
dataflow = builder.to_dataflow()

In [11]:
df = dataflow.to_pandas_dataframe()
df

,DATE,date_timerange,REPORTTPYE,HOURLYDRYBULBTEMPF,HOURLYRelativeHumidity,HOURLYWindSpeed
0,1/1/2015 0:54,"Jan 1, 2015 | 12AM-2AM",FM-15,22,50,10
1,1/1/2015 1:00,"Jan 1, 2015 | 12AM-2AM",FM-12,22,50,10
2,1/1/2015 1:54,"Jan 1, 2015 | 12AM-2AM",FM-15,22,50,10
3,1/1/2015 2:54,"Jan 1, 2015 | 2AM-4AM",FM-15,22,50,11
4,1/1/2015 3:54,"Jan 1, 2015 | 2AM-4AM",FM-15,24,46,13
5,1/1/2015 4:00,"Jan 1, 2015 | 4AM-6AM",FM-12,24,46,13
6,1/1/2015 4:54,"Jan 1, 2015 | 4AM-6AM",FM-15,22,52,15
7,1/1/2015 5:54,"Jan 1, 2015 | 4AM-6AM",FM-15,23,48,17
8,1/1/2015 6:54,"Jan 1, 2015 | 6AM-8AM",FM-15,23,50,14
9,1/1/2015 7:00,"Jan 1, 2015 | 6AM-8AM",FM-12,23,50,14
